## Task 3.1 Convergence and attractors

In [70]:
import numpy as np

#### Initialize some data from lab instructions

In [71]:
x1=np.array([-1, -1, 1, -1, 1, -1, -1, 1])
x2=np.array([-1, -1, -1, -1, -1, 1, -1, -1])
x3=np.array([-1, 1, 1, -1, -1, 1, -1, 1])

x1d=np.array([ 1, -1, 1, -1, 1, -1, -1, 1])
x2d=np.array([ 1, 1, -1, -1, -1, 1, -1, -1])
x3d=np.array([ 1, 1, 1, -1, 1, 1, -1, 1,])

#### Synchronous Hopfield - note, all data is in row format

In [104]:
class Hopfield_net():
    def __init__(self, data, sync=True, scale=False):
        self.data = data
        self.W = np.zeros((data.shape[1], data.shape[1]))
        self.scale = scale
        self.hebian_learn()

    def hebian_learn(self):
        for i in range(self.data.shape[0]):
            self.W += np.outer(self.data[i], self.data[i])
        if self.scale:
            self.W /= self.data.shape[0]
    
    def recall_sync(self, dp):
        recall_prev = dp
        recall = np.sign(np.matmul(dp, self.W))
        # Numpy sign sets sign of 0 to 0 - change it to 1
        recall = np.where(recall == 0, 1, recall)
        while not np.array_equal(recall, recall_prev):
            recall_prev = recall
            recall = np.sign(np.matmul(recall, self.W))
            recall = np.where(recall == 0, 1, recall)
        return recall.astype(int)

    def recall_seq(self, dp):
        pass

def test_recall_sync(data, data_dist, test_net=None):
    if test_net is None:
        test_net = Hopfield_net(data)
    for i in range(data.shape[0]):
        recalled = test_net.recall_sync(data_dist[i])
        did_recall = np.array_equal(data[i], recalled)
        print("x{} recalled properly: {}".format(i + 1, did_recall))
        if not did_recall:
            print("\torig: {}".format(np.array2string(data[i])))
            print("\tdist: {}".format(np.array2string(data_dist[i])))
            print("\trecl: {}".format(np.array2string(recalled)))
        

##### Some sanity checks

In [105]:
data = np.vstack((x1, x2, x3))
test_recall_sync(data, data)

x1 recalled properly: True
x2 recalled properly: True
x3 recalled properly: True


###  Actual 3.1

#### Check convergence of distorted data points

In [106]:
data = np.vstack((x1, x2, x3))
data_dist = np.vstack((x1d, x2d, x3d))
test_recall_sync(data, data_dist)

x1 recalled properly: True
x2 recalled properly: False
	orig: [-1 -1 -1 -1 -1  1 -1 -1]
	dist: [ 1  1 -1 -1 -1  1 -1 -1]
	recl: [-1  1 -1 -1 -1  1 -1 -1]
x3 recalled properly: True


#### Check for all attractors

In [107]:
# Please change this if you get a better idea than to brute force it with all possible input vectors
def dp_builder(dim):
    vec = np.zeros(dim)
    return dp_builder_h(dim, vec, 0)
def dp_builder_h(dim, curr_vec, curr_ind):
    # Need to copy so vec2 does not mess this up later
    vec1 = np.copy(curr_vec)
    vec1[curr_ind] = 1
    # Don't need to copy since no one else will use curr_vec
    vec2 = curr_vec
    vec2[curr_ind] = -1
    # Base case, reached last index of vector
    if dim == curr_ind + 1:
        return np.vstack((vec1, vec2))
    else:
        return np.vstack((
            dp_builder_h(dim, vec1, curr_ind + 1),
            dp_builder_h(dim, vec2, curr_ind + 1)
        ))        

In [108]:
data = np.vstack((x1, x2, x3))
test_net = Hopfield_net(data)
all_dp = dp_builder(x1.shape[0])
attractors = np.zeros(all_dp.shape)

# Brute force it - recall for every possible input vector
for i in range(all_dp.shape[0]):
    attractors[i] = test_net.recall_sync(all_dp[i])

# Remove duplicates
attractors = np.unique(attractors, axis=0)
print(attractors)
print("In total {} attractors".format(attractors.shape[0]))
    

[[-1. -1. -1. -1. -1.  1. -1. -1.]
 [-1. -1. -1. -1.  1. -1. -1. -1.]
 [-1. -1.  1. -1. -1.  1. -1.  1.]
 [-1. -1.  1. -1.  1. -1. -1.  1.]
 [-1. -1.  1. -1.  1.  1. -1.  1.]
 [-1.  1. -1. -1. -1.  1. -1. -1.]
 [-1.  1.  1. -1. -1.  1. -1.  1.]
 [-1.  1.  1. -1.  1. -1. -1.  1.]
 [ 1. -1. -1.  1.  1. -1.  1. -1.]
 [ 1.  1. -1.  1. -1.  1.  1. -1.]
 [ 1.  1. -1.  1.  1. -1.  1. -1.]
 [ 1.  1. -1.  1.  1.  1.  1. -1.]
 [ 1.  1.  1.  1. -1.  1.  1.  1.]
 [ 1.  1.  1.  1.  1. -1.  1.  1.]]
In total 14 attractors


#### Testing even more distorted patterns

In [112]:
# Flip first 5 
x1_superd=np.copy(x1)
for i in range(x1.size // 2 + 1):
    x1_superd[i] = x1_superd[i] * -1

test_recall_sync(x1.reshape((1, -1)), x1_superd.reshape((1, -1)), test_net)
    

x1 recalled properly: False
	orig: [-1 -1  1 -1  1 -1 -1  1]
	dist: [ 1  1 -1  1 -1 -1 -1  1]
	recl: [ 1  1  1  1  1 -1  1  1]
